In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@40b33105

In [2]:
import spark.implicits._

case class QueuePerson(person_id: Int, person_name: String, weight: Int, turn: Int)

val queuePerson: Seq[(Int, String, Int, Int)] = Seq(
  (5, "Alice", 250, 1),
  (4, "Bob", 175, 5),
  (3, "Alex", 350, 2),
  (6, "John Cena", 400, 3),
  (1, "Winston", 500, 6),
  (2, "Marie", 200, 4)
)

val queueDS = queuePerson.toDF("person_id", "person_name", "weight", "turn").as[QueuePerson]


import spark.implicits._


defined class QueuePerson
queuePerson: Seq[(Int, String, Int, Int)] = List(
  (5, "Alice", 250, 1),
  (4, "Bob", 175, 5),
  (3, "Alex", 350, 2),
  (6, "John Cena", 400, 3),
  (1, "Winston", 500, 6),
  (2, "Marie", 200, 4)
)
queueDS: Dataset[QueuePerson] = [person_id: int, person_name: string ... 2 more fields]

There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

Note that only one person can board the bus at any given turn.

The result format is in the following example.


| person_name |
|-------------|
| John Cena   |

In [3]:
val windowSpec = Window.orderBy(col("turn"))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@47d917fa

In [4]:
val sumDF = queueDS.withColumn("sum", sum(col("weight")).over(windowSpec))
    .filter(col("sum") <= 1000)
val lastWeightDF = sumDF.select("sum").agg(max(col("sum")).as("sum"))
sumDF.join(lastWeightDF, Seq("sum"), "inner")
    .select("person_name")
    .show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



sumDF: Dataset[Row] = [person_id: int, person_name: string ... 3 more fields]
lastWeightDF: DataFrame = [sum: bigint]

In [5]:
spark.stop()